<a href="https://colab.research.google.com/github/joydipb01/Stock_Market_Prediction/blob/main/Stock_Market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">STOCK MARKET PREDICTION<BR>USING NLP AND DATA ANALYTICS</h1>
<h5>By: Joydip Kishore Bhattacharyya</h5>

In [ ]:
!pip install textblob

In [ ]:
!pip install lightgbm

In [ ]:
!pip install wordcloud

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

from yellowbrick.classifier import ROCAUC

import datetime

from textblob import TextBlob
import lightgbm

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as vader
from pandas.plotting import andrews_curves
from wordcloud import WordCloud

import warnings
warnings.simplefilter('ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


<h1>Data Analysis</h1>

In [ ]:
stock_price=pd.read_csv("^BSEN.csv")
stock_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,"May 18, 2021","49,986.68","50,313.25","49,959.20","50,193.33","50,193.33",-
1,"May 17, 2021","48,990.70","49,628.42","48,923.13","49,580.73","49,580.73","20,600"
2,"May 14, 2021","48,898.93","48,898.93","48,473.43","48,732.55","48,732.55","18,200"
3,"May 12, 2021","49,171.28","49,171.28","48,550.72","48,690.80","48,690.80","17,400"
4,"May 11, 2021","49,066.45","49,304.47","48,988.18","49,161.81","49,161.81","15,000"


In [ ]:
stock_price.shape

(97, 7)

In [ ]:
stock_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 1 to 99
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       97 non-null     datetime64[ns]
 1   Open       97 non-null     float64       
 2   High       97 non-null     float64       
 3   Low        97 non-null     float64       
 4   Close      97 non-null     float64       
 5   Adj Close  97 non-null     float64       
 6   Volume     97 non-null     float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 6.1 KB


In [ ]:
stock_price=stock_price.dropna()

In [ ]:
stock_price['Volume'] = np.where(stock_price['Volume']=='-', np.nan, stock_price['Volume'])

In [ ]:
stock_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
1,"May 17, 2021","48,990.70","49,628.42","48,923.13","49,580.73","49,580.73","20,600"
2,"May 14, 2021","48,898.93","48,898.93","48,473.43","48,732.55","48,732.55","18,200"
3,"May 12, 2021","49,171.28","49,171.28","48,550.72","48,690.80","48,690.80","17,400"
4,"May 11, 2021","49,066.45","49,304.47","48,988.18","49,161.81","49,161.81","15,000"
5,"May 10, 2021","49,496.05","49,617.47","49,412.05","49,502.41","49,502.41","12,200"


In [ ]:
stock_price['Date']=pd.to_datetime(stock_price['Date'])

In [ ]:
for i in stock_price.columns:
    if i !='Date':
        stock_price[i]=stock_price[i].str.replace(',', '').astype(float)

In [ ]:
stock_price.sort_values(by=['Date'])

,Date,Open,High,Low,Close,Adj Close,Volume
99,2020-12-18,47026.02,47026.02,46630.31,46960.69,46960.69,12800.0
98,2020-12-21,46932.18,47055.69,44923.08,45553.96,45553.96,24400.0
97,2020-12-22,45529.61,46080.18,45112.19,46006.69,46006.69,25100.0
96,2020-12-23,46072.30,46513.32,45899.10,46444.18,46444.18,10500.0
95,2020-12-24,46743.49,47053.40,46539.02,46973.54,46973.54,13700.0
...,...,...,...,...,...,...,...
5,2021-05-10,49496.05,49617.47,49412.05,49502.41,49502.41,12200.0
4,2021-05-11,49066.45,49304.47,48988.18,49161.81,49161.81,15000.0
3,2021-05-12,49171.28,49171.28,48550.72,48690.80,48690.80,17400.0
2,2021-05-14,48898.93,48898.93,48473.43,48732.55,48732.55,18200.0


In [ ]:
stock_price["symb"]="BSE"
stock_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume,symb
1,2021-05-17,48990.70,49628.42,48923.13,49580.73,49580.73,20600.0,BSE
2,2021-05-14,48898.93,48898.93,48473.43,48732.55,48732.55,18200.0,BSE
3,2021-05-12,49171.28,49171.28,48550.72,48690.80,48690.80,17400.0,BSE
4,2021-05-11,49066.45,49304.47,48988.18,49161.81,49161.81,15000.0,BSE
5,2021-05-10,49496.05,49617.47,49412.05,49502.41,49502.41,12200.0,BSE


In [ ]:
stock_price.columns = ['date', 'open', 'high', 'low', 'close', 'adj', 'volume', 'symb']
stock_price.head()

,date,open,high,low,close,adj,volume,symb
1,2021-05-17,48990.70,49628.42,48923.13,49580.73,49580.73,20600.0,BSE
2,2021-05-14,48898.93,48898.93,48473.43,48732.55,48732.55,18200.0,BSE
3,2021-05-12,49171.28,49171.28,48550.72,48690.80,48690.80,17400.0,BSE
4,2021-05-11,49066.45,49304.47,48988.18,49161.81,49161.81,15000.0,BSE
5,2021-05-10,49496.05,49617.47,49412.05,49502.41,49502.41,12200.0,BSE


In [ ]:
stock_price.drop(['adj'], axis=1, inplace=True)
stock_price.head()

,date,open,high,low,close,volume,symb
1,2021-05-17,48990.70,49628.42,48923.13,49580.73,20600.0,BSE
2,2021-05-14,48898.93,48898.93,48473.43,48732.55,18200.0,BSE
3,2021-05-12,49171.28,49171.28,48550.72,48690.80,17400.0,BSE
4,2021-05-11,49066.45,49304.47,48988.18,49161.81,15000.0,BSE
5,2021-05-10,49496.05,49617.47,49412.05,49502.41,12200.0,BSE


In [ ]:
stock_price["price_change"]=stock_price["close"].diff()
stock_price["price_change"][1]=0.0
stock_price.head()

,date,open,high,low,close,volume,symb,price_change
1,2021-05-17,48990.70,49628.42,48923.13,49580.73,20600.0,BSE,0.00
2,2021-05-14,48898.93,48898.93,48473.43,48732.55,18200.0,BSE,-848.18
3,2021-05-12,49171.28,49171.28,48550.72,48690.80,17400.0,BSE,-41.75
4,2021-05-11,49066.45,49304.47,48988.18,49161.81,15000.0,BSE,471.01
5,2021-05-10,49496.05,49617.47,49412.05,49502.41,12200.0,BSE,340.60


In [ ]:
stock_price = stock_price[['symb', 'date', 'close', 'high', 'low', 'open', 'volume', 'price_change']]
stock_price.head()

,symb,date,close,high,low,open,volume,price_change
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60


<h1>NLP</h1>

In [ ]:
increase, decrease=stock_price[['symb', 'price_change']].copy(), stock_price[['symb', 'price_change']].copy()
increase.loc['price_change'] = increase.loc[(increase['price_change'] < 0), 'price_change'] = 0
decrease.loc['price_change'] = decrease.loc[(decrease['price_change'] > 0), 'price_change'] = 0
decrease['price_change']=decrease['price_change'].abs()
einc=increase.groupby('symb')['price_change'].transform(lambda x: x.ewm(span = 14).mean())
edec=decrease.groupby('symb')['price_change'].transform(lambda x: x.ewm(span = 14).mean())
rs=einc/edec
rsi=100.0-(100.0/(1.0+rs))
stock_price['price_increase']=increase['price_change']
stock_price['price_decrease']=decrease['price_change']
stock_price['rsi']=rsi
stock_price.head()

,symb,date,close,high,low,open,volume,price_change,price_increase,price_decrease,rsi
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00,0.00,0.00,NaN
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18,0.00,848.18,0.000000
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75,0.00,41.75,0.000000
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01,471.01,0.00,41.162461
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60,340.60,0.00,56.204153


In [ ]:
stock_price['rsi'][1]=0
stock_price.head()

,symb,date,close,high,low,open,volume,price_change,price_increase,price_decrease,rsi
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00,0.00,0.00,0.000000
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18,0.00,848.18,0.000000
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75,0.00,41.75,0.000000
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01,471.01,0.00,41.162461
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60,340.60,0.00,56.204153


In [ ]:
low, high=stock_price[['symb', 'low']].copy(), stock_price[['symb', 'high']].copy()
low=low.groupby('symb')['low'].transform(lambda x: x.rolling(window = 14).min())
high=high.groupby('symb')['high'].transform(lambda x: x.rolling(window = 14).max())
so=100*((stock_price['close']-low)/(high-low))
stock_price['mov_avg_low']=low
stock_price['mov_avg_high']=high
stock_price['stoch_osc']=so
stock_price.head()

,symb,date,close,high,low,open,volume,price_change,price_increase,price_decrease,rsi,mov_avg_low,mov_avg_high,stoch_osc
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00,0.00,0.00,0.000000,NaN,NaN,NaN
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18,0.00,848.18,0.000000,NaN,NaN,NaN
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75,0.00,41.75,0.000000,NaN,NaN,NaN
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01,471.01,0.00,41.162461,NaN,NaN,NaN
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60,340.60,0.00,56.204153,NaN,NaN,NaN


In [ ]:
for i in ['mov_avg_low', 'mov_avg_high', 'stoch_osc']:
    stock_price[i]=stock_price[i].fillna(0)
stock_price.head()

,symb,date,close,high,low,open,volume,price_change,price_increase,price_decrease,rsi,mov_avg_low,mov_avg_high,stoch_osc
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18,0.00,848.18,0.000000,0.0,0.0,0.0
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75,0.00,41.75,0.000000,0.0,0.0,0.0
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01,471.01,0.00,41.162461,0.0,0.0,0.0
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60,340.60,0.00,56.204153,0.0,0.0,0.0


In [ ]:
william_r=((high-stock_price['close'])/(high-low))*-100
stock_price['william_r_percent']=william_r
stock_price.head()

,symb,date,close,high,low,open,volume,price_change,price_increase,price_decrease,rsi,mov_avg_low,mov_avg_high,stoch_osc,william_r_percent
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,NaN
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18,0.00,848.18,0.000000,0.0,0.0,0.0,NaN
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75,0.00,41.75,0.000000,0.0,0.0,0.0,NaN
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01,471.01,0.00,41.162461,0.0,0.0,0.0,NaN
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60,340.60,0.00,56.204153,0.0,0.0,0.0,NaN


In [ ]:
stock_price['william_r_percent']=stock_price['william_r_percent'].fillna(0)
stock_price.head()

,symb,date,close,high,low,open,volume,price_change,price_increase,price_decrease,rsi,mov_avg_low,mov_avg_high,stoch_osc,william_r_percent
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.0
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18,0.00,848.18,0.000000,0.0,0.0,0.0,0.0
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75,0.00,41.75,0.000000,0.0,0.0,0.0,0.0
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01,471.01,0.00,41.162461,0.0,0.0,0.0,0.0
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60,340.60,0.00,56.204153,0.0,0.0,0.0,0.0


In [ ]:
e_26=stock_price.groupby('symb')['close'].transform(lambda x: x.ewm(span = 26).mean())
e_12=stock_price.groupby('symb')['close'].transform(lambda x: x.ewm(span = 12).mean())
macd=e_12-e_26
e_9_macd=macd.ewm(span=9).mean()
stock_price['macd']=macd
stock_price['macd_ewm_9period']=e_9_macd
stock_price.head()

,symb,date,close,high,low,open,volume,price_change,price_increase,price_decrease,rsi,mov_avg_low,mov_avg_high,stoch_osc,william_r_percent,macd,macd_ewm_9period
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18,0.00,848.18,0.000000,0.0,0.0,0.0,0.0,-19.029679,-10.572044
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75,0.00,41.75,0.000000,0.0,0.0,0.0,0.0,-25.542254,-16.707376
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01,471.01,0.00,41.162461,0.0,0.0,0.0,0.0,-10.872688,-14.730856
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60,340.60,0.00,56.204153,0.0,0.0,0.0,0.0,10.880690,-7.112000


In [ ]:
stock_price['pct_change_price']=stock_price.groupby('symb')['close'].transform(lambda x: x.pct_change(periods=9))
stock_price['pct_change_price']=stock_price['pct_change_price'].fillna(0)
stock_price.head()

,symb,date,close,high,low,open,volume,price_change,price_increase,price_decrease,rsi,mov_avg_low,mov_avg_high,stoch_osc,william_r_percent,macd,macd_ewm_9period,pct_change_price
1,BSE,2021-05-17,49580.73,49628.42,48923.13,48990.70,20600.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,BSE,2021-05-14,48732.55,48898.93,48473.43,48898.93,18200.0,-848.18,0.00,848.18,0.000000,0.0,0.0,0.0,0.0,-19.029679,-10.572044,0.0
3,BSE,2021-05-12,48690.80,49171.28,48550.72,49171.28,17400.0,-41.75,0.00,41.75,0.000000,0.0,0.0,0.0,0.0,-25.542254,-16.707376,0.0
4,BSE,2021-05-11,49161.81,49304.47,48988.18,49066.45,15000.0,471.01,471.01,0.00,41.162461,0.0,0.0,0.0,0.0,-10.872688,-14.730856,0.0
5,BSE,2021-05-10,49502.41,49617.47,49412.05,49496.05,12200.0,340.60,340.60,0.00,56.204153,0.0,0.0,0.0,0.0,10.880690,-7.112000,0.0


In [ ]:
def on_bal_vol(grp):
    vol=grp['volume']
    change=grp['close'].diff()
    prev_on_bal_vol=0
    on_bal_vols=[]
    for i, j in zip(change, vol):
        if i>0:
            curr_on_bal_vol=prev_on_bal_vol+j
        elif i<0:
            curr_on_bal_vol=prev_on_bal_vol-j
        else:
            curr_on_bal_vol=prev_on_bal_vol
        prev_on_bal_vol=curr_on_bal_vol
        on_bal_vols.append(curr_on_bal_vol)
    return pd.Series(on_bal_vols, index=grp.index)

In [ ]:
##on_bal_vol_grps=stock_price.groupby("symb").apply(on_bal_vol)
##on_bal_vol_grps=on_bal_vol_grps.transpose()
